In [1]:
import predictpostures
reload(predictpostures)

typed= []
pred = []
for i in range(4, 5):
    print "user "+ str(i)
    a, b = predictpostures.run(i)
    typed += a    
    pred += b
    

user 4


In [57]:
a = np.array([[1,2],[3,4],[5,6],[7,8]])

print np.delete(a, np.s_[1:2],0)

[[1 2]
 [5 6]
 [7 8]]


In [2]:
import numpy as np

print np.array(typed).shape
print np.array(pred).shape
print    
print np.mean(np.array(typed))
print np.mean(np.array(pred)) 

(42,)
(42,)

9.83012751718
10.1376851864


In [6]:
import numpy as np
from scipy import stats

#numpy.set_printoptions(precision=2)    
 

#a = np.array([23.07, 17.14, 26.19, 11.41, 4.26, 13.19, 19.93, 13.79, 5.41, 18.74, 18.89, 19.11, 16.55, 18.16, 20.91])
#b = np.array([23.04, 12.60, 27.05, 12.71, 7.94, 7.66, 25.01, 8.50, 5.94, 18.92, 14.41, 15.78, 14.83, 17.81, 16.82])

a = np.array(typed)
b = np.array(pred)

print stats.ttest_rel(a, b)

(4.0085327854478319, 6.8423771530603123e-05)


In [1]:
import numpy as np
import dataIO
from sklearn import cross_validation, linear_model
reload(dataIO)
import Levenshtein

def error_rate(a, b):
    msd = Levenshtein.distance(a,b)
    
    return 100.0*msd/max(len(a), len(b))

def learn_offset(points, targets):
    regr = linear_model.LinearRegression()
    regr.fit(points, targets)
    
    return regr

    

In [10]:
### POOLED

import numpy as np
import ast
import dataIO
reload(dataIO)

userId = 11
locations, bod, targets_x, targets_y, y, touch_centers = dataIO.process_twohand(userId)

locations = np.array(locations)
targets_x = np.array(targets_x)
targets_y = np.array(targets_y)

regr_x = learn_offset(locations, targets_x)
regr_y = learn_offset(locations, targets_y)

with open("/home/dimitar/Desktop/Python/Level5Project/Loggingapp/dataset.txt") as f:
    pool = f.read().splitlines()

#def read(userId):
req_sentences = []
typed_sentences = []
pred_sentences = []
#for i in range(13, 25):
for i in [15, 17, 21]:
    column_index = 0
    if i in [15, 17, 21]:
        column_index = 1

    with open("/home/dimitar/Desktop/Python/experiment/results/"
               +str(userId)+"_"+str(i)+"up.txt") as f:
        lines = f.read().splitlines()
        touches = map(lambda x: x.split('\t'), lines[1:])

    req_string = ''.join(np.array(touches)[:,column_index]) 
    locations = map(lambda x: list(ast.literal_eval(x)),
                            np.array(touches)[:,(column_index+2)])
   
    for sentence in pool: 
        index = req_string.find(sentence)
        if index!=-1:
            req_sentences.append(sentence)
            typed_locations = np.array(locations[index:index+len(sentence)])
            
            pred_x = regr_x.predict(typed_locations)
            pred_y = regr_y.predict(typed_locations)
            pred = typed_locations + np.dstack((pred_x, pred_y))[0]
            
            typed_sentences.append(dataIO.typed_string(typed_locations))
            pred_sentences.append(dataIO.typed_string(pred))

error_typed = []
error_pred = []
for i in range(len(req_sentences)):    
    req_sentence = req_sentences[i]
    typed_sentence = typed_sentences[i]
    pred_sentence = pred_sentences[i]
    error_typed.append(error_rate(req_sentence, typed_sentence))
    error_pred.append(error_rate(req_sentence, pred_sentence))

print
print ("Mean error rate before: %.2f %%"% np.mean(np.array(error_typed)))
print ("Mean error rate after: %.2f %%"% np.mean(np.array(error_pred)))

TypeError: distance expected two Strings or two Unicodes